In [1]:
from src.data_manager import DataManager

sheet_id = "***REMOVED***"

data_manager = DataManager(sheet_id, sheet_name="***REMOVED***")

df = data_manager.excel_to_dataframe("***REMOVED***")

In [2]:
from datetime import datetime, timedelta
import pandas as pd

from src.parse_excel_line import ParseExcelLine

line_parser = ParseExcelLine(df, datetime(2024, 1, 15))
line_parser.parse()

In [3]:
from src.calendar_manager import CalendarManager

cal = CalendarManager(line_parser.course_list)
cal.browse_course_list()